### TOPUP MAMA - PRODUCT RECOMMENDATION

##### This is a simple recommender system that takes in Product categories as items and Customer_ID as users. 
##### The ratings were too few to employ a collaborative approach and therefore a content-based approach is employed.

#### IMPORTING MODULES

In [1]:
import numpy as np
import pandas as pd
import sklearn
import tensorflow as tf 

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')
import seaborn as sns
from pandas_profiling import ProfileReport
import tkinter
matplotlib.use('TkAgg')

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler


from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('df.csv')

data.head()

C:\Users\MUKIRI\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (2,4,5,9,11,12,13,18,19,23,24,25,26,28,29,30,32,36,37,38,40,41,42,44,49,50,51,54,55,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,Customer_ID,Last_Used_Platform,Is_Blocked,Created_At,Language,Outstanding_Amount,Loyalty_Points,Number_of_Employees,Upload_restuarant_location,...,Delivery_Charges,Discount_y,Sub_Total_y,Payment_Type,Task_Category,Earning,Pricing,Unnamed: 34,Unnamed: 35,Country
0,0,3365019.0,WEB,0.0,2021-04-21T07:15:56.000Z,en,0.0,68.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kenya
1,1,3365019.0,WEB,0.0,2021-04-21T07:15:56.000Z,en,0.0,68.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kenya
2,2,3365019.0,WEB,0.0,2021-04-21T07:15:56.000Z,en,0.0,68.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kenya
3,3,3365019.0,WEB,0.0,2021-04-21T07:15:56.000Z,en,0.0,68.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kenya
4,4,3365770.0,WEB,0.0,2021-04-21T10:23:50.000Z,en,0.0,55.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kenya


In [3]:
num_items = data.Category_Name.nunique()
num_users = data.Customer_ID.nunique()
print(num_items, num_users)

14 702


#### Fill Missing Values

In [4]:
data.Category_Name.isnull().sum()

53008

In [5]:
data.Customer_ID.isnull().sum()

53002

#### Encoding Categorical Data

Opt for the LabelEncoder in this case.

In [6]:
from sklearn.preprocessing import LabelEncoder
cols = ['Category_Name']
lp = LabelEncoder()
for col in cols:
    data[col] = lp.fit_transform(data[col])

In [7]:
import pickle
filehandler = open("lp.obj", "wb")
pickle.dump(lp, filehandler)
filehandler.close()

#### Creating our Matrix

In [8]:
#CREATE A MATRIX FOR THE ABOVE
column_names = ['Customer_ID', 'Category_Name']
features = data[column_names]
df = pd.DataFrame(features, columns = column_names)
df.head()

,Customer_ID,Category_Name
0,3365019.0,3
1,3365019.0,4
2,3365019.0,4
3,3365019.0,3
4,3365770.0,3


In [9]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [10]:
df.fillna(0, inplace=True)

In [11]:
df.dtypes

Customer_ID      float64
Category_Name      int32
dtype: object

In [12]:
df = df.sort_values(by='Customer_ID')

In [13]:
df.set_index(['Customer_ID'])

,Category_Name
Customer_ID,
0.0,14
0.0,14
0.0,14
0.0,14
0.0,14
...,...
5188263.0,4
5188263.0,4
5188263.0,4


In [14]:
df.head()

,Customer_ID,Category_Name
33336,0.0,14
43488,0.0,14
43489,0.0,14
43490,0.0,14
43491,0.0,14


In [15]:
df = df.sort_values(by='Category_Name')

In [16]:
df.head()

,Customer_ID,Category_Name
58580,4876172.0,0
58578,4876172.0,0
58579,4876172.0,0
11840,4934058.0,1
3533,4054645.0,1


#### Feature Scaling

In [17]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
df_scaled = sc.fit_transform(df)

In [18]:
df.head()

,Customer_ID,Category_Name
58580,4876172.0,0
58578,4876172.0,0
58579,4876172.0,0
11840,4934058.0,1
3533,4054645.0,1


In [19]:
#df = df.reset_index()

#### Split the data as the matrix is too large

In [20]:
df = df.iloc[1:1000, :].values

####  The Model

In [21]:
from sklearn.metrics.pairwise import linear_kernel

In [22]:
# Compute the cosine similarity matrix
cosine_sim = linear_kernel(df)

In [23]:
cosine_sim.shape

(999, 999)

In [24]:
cosine_sim[1]

array([2.37770534e+13, 2.37770534e+13, 2.40593155e+13, 1.97711464e+13,
       2.02535802e+13, 2.02535802e+13, 1.97711464e+13, 2.02535802e+13,
       1.97684450e+13, 2.02693059e+13, 2.40991684e+13, 2.02700178e+13,
       2.23725842e+13, 2.24579074e+13, 2.40991684e+13, 2.40991684e+13,
       2.40623436e+13, 2.40991684e+13, 2.23725842e+13, 2.23725842e+13,
       2.40991684e+13, 2.23725842e+13, 2.24579074e+13, 1.66366699e+13,
       2.23725842e+13, 2.41306441e+13, 2.02700178e+13, 2.24579074e+13,
       1.97711464e+13, 1.66623088e+13, 2.31373337e+13, 2.22832332e+13,
       1.81975082e+13, 2.22832332e+13, 2.27917838e+13, 2.01072561e+13,
       2.01072561e+13, 2.40567652e+13, 2.40567652e+13, 2.25710151e+13,
       1.84994213e+13, 1.98076689e+13, 2.25710151e+13, 2.19160379e+13,
       2.40567652e+13, 1.89218538e+13, 1.84994213e+13, 2.18885120e+13,
       1.97842877e+13, 1.81975082e+13, 1.89218538e+13, 1.71617556e+13,
       1.97711464e+13, 2.40593155e+13, 2.01830854e+13, 2.40623436e+13,
      